In [18]:
import os

In [19]:
patent_abstract = """
Method to improve blowthrough and EGR via split exhaust
Abstract
Methods and systems are provided for a boosted engine having a split exhaust system. In one example, a method comprises directing exhaust from a first cylinder group to one or more of a pre-compressor location, a post-compressor location, and an exhaust turbine, and directing exhaust from a second cylinder group to one or more of the pre-compressor location, and the exhaust turbine. Engine efficiency and knock control may be enhanced by directing exhaust gases to different locations based on engine operating conditions.
"""


In [20]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=patent_abstract
    )


CreateEmbeddingResponse(data=[Embedding(embedding=[-0.04720533266663551, 0.016175443306565285, -0.008849315345287323, -0.029412347823381424, -0.015083600766956806, 0.027484772726893425, 0.007986624725162983, -0.02238950878381729, -0.014517460018396378, -0.008586464449763298, -0.014342226088047028, 0.0005151710356585681, 0.004390958696603775, 0.048822879791259766, -0.009489593096077442, -0.0049436199478805065, 0.021324625238776207, -0.007460922934114933, -0.00030897624674253166, 0.008754958398640156, -0.0031862251926213503, 0.0017455999040976167, -0.018534362316131592, -0.01736164279282093, -0.0025240429677069187, -0.0027346608694642782, 0.027579130604863167, -0.01753687672317028, 0.002684112638235092, -0.005728802643716335, 0.0025105634704232216, -0.004542603623121977, -0.026258135214447975, -0.013533453457057476, 0.003639474743977189, -0.01908702217042446, -0.010224227793514729, 0.004724577069282532, 0.013573892414569855, 0.0018298469949513674, 0.0302211195230484, -0.00913238525390625

In [21]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Patents",
                                                      embedding_function=embedding_function)

In [22]:
vector_store.add("patent_info", documents=patent_abstract)

Add of existing embedding ID: patent_info
Insert of existing embedding ID: patent_info


In [23]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
#tru.reset_database() #Reset the database. Clears all tables.


In [24]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"The following abstract descripes a concept for a novel invention: \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please execute the following instructions: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [25]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [26]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG Key Words',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001C46A5D0280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001C46A5D0310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001C46A5D01F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [27]:
with tru_rag as recording:
    rag.query("Name 5 key words based on this abstract, that I can use for the search in a patent database. Optimize the key words to get back more results. Result as python string.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [28]:
tru.get_leaderboard(app_ids=["RAG Key Words"])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
RAG Key Words,1.0,1.0,0.95,3.333333,0.0


In [29]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Classifications',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001C46A5D0280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001C46A5D0310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001C46A5D01F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [30]:
with tru_rag as recording:
    rag.query("Name 5 CPC classifications based on this abstract, that I can use for the search in a patent database. \
Please give me a python string for the codes of the 5 most relevant \
CPC classifications to a possible patent.")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [31]:
tru.get_leaderboard(app_ids=["RAG Classifications"])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
RAG Classifications,0.21,1.0,0.45,3.666667,0.0


In [32]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Patent Comparison',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

Function <function RAG_from_scratch.generate_completion at 0x000001C46A5D0280> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.query at 0x000001C46A5D0310> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG_from_scratch.retrieve at 0x000001C46A5D01F0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG_from_scratch object at 0x000001C46C0BF940> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.


In [33]:
with tru_rag as recording:
    rag.query(f"""
'The following texts are abstracts from patent specifications. Your task is to compare the novel invention from above to all the other abstracts from below. \n
It is important that you focus on comparing the concepts that the abstracts describe, not the way they are written. \n
Rank the remaining abstracts on how well they match with the Testing Abstract by giving them a rating from 0 to 10 points. \n        
0 meaning they have absolutely nothing in common and 10 meaning they basically describe the exact same idea.\n  
Your output should be a python dictionary with the title "comparison", each element hast the Abstract number as key and the rating as value.\n        
I want to convert your output string to an actual dictionary, so make sure the formatting is right.\n\n        
        
US10161334B2: "Methods and systems are provided for operating a split exhaust engine system that provides blowthrough air and exhaust gas recirculation to an intake passage via a first exhaust manifold and exhaust gas to an exhaust passage via a second exhaust manifold. In one example, in response to a request to shut down the split exhaust engine system, an intake throttle may be closed and a first valve disposed in a secondary flow passage coupled between the intake manifold, downstream of the intake throttle, and a first exhaust manifold coupled to a first set of exhaust valves, may be opened. As a result, unburned hydrocarbons may be routed to a catalyst disposed in the exhaust passage."\nUS10316771B2: "Methods and systems are provided for operating a split exhaust engine system that provides blowthrough air and exhaust gas recirculation to an intake passage via a first exhaust manifold and exhaust gas to an exhaust passage via a second exhaust manifold. In one example, during a cold start, a method may include adjusting a position of a first valve disposed in an exhaust gas recirculation (EGR) passage based on an engine operating condition, the EGR passage coupled between the first exhaust manifold coupled to a first set of exhaust valves and the intake passage, upstream of a compressor. As one example, the engine operating condition may include an engine temperature or a temperature of a catalyst disposed in the exhaust passage."\nUS10882511B2: "Methods and systems are provided for operating a split exhaust engine system that provides blowthrough air and exhaust gas recirculation to an intake passage via a first exhaust manifold and exhaust gas to an exhaust passage via a second exhaust manifold. In one example, the engine system may be installed in a hybrid vehicle, and, in response to a request to restart the engine while the vehicle is being propelled via motor torque only, the engine may be rotated unfueled via the motor torque at less than cranking speed while at least partially opening a valve disposed in a passage coupled between the first exhaust manifold and the intake passage. In another example, in response to the request to restart the engine, all exhaust valves of a second set of exhaust valves coupled to the second exhaust manifold may be deactivated."\nUS10557425B2: "Methods and systems are provided for operating a split exhaust engine system that provides blowthrough air and exhaust gas recirculation to an intake passage via a first exhaust manifold and exhaust gas to an exhaust passage via a second exhaust manifold. In one example, in response to an electric motor driving an electric compressor positioned upstream of a turbocharger compressor disposed in the intake passage, a position of a valve in an exhaust gas recirculation (EGR) passage coupled between the intake passage and the first exhaust manifold may be adjusted based on a pressure in the first exhaust manifold."\nUS10590875B2: "Methods and systems are provided for operating a split exhaust engine system that provides blowthrough air and exhaust gas recirculation to an intake passage via a first exhaust manifold and exhaust gas to an exhaust passage via a second exhaust manifold. In one example, a first set of exhaust valves coupled to the first exhaust manifold may be operated at a different timing than a second set of exhaust valves coupled to the second exhaust manifold. Further, a position of a first valve positioned in a first passage coupled between the intake passage and the first exhaust manifold and/or a timing of the first set of exhaust valves may be diagnosed based on an output of a pressure sensor positioned in the first exhaust manifold."\nCN108204303B: "Methods and systems for operating a split-flow exhaust engine system that provides blow-through air and exhaust gas recirculation to an intake port via a first exhaust manifold and exhaust gas to an exhaust port via a second exhaust manifold are provided. In one example, a method includes supplying air to an exhaust system at a location downstream of an emission control device via a first exhaust manifold, the air not yet participating in combustion in the engine, the first exhaust manifold in fluid communication with a first exhaust valve of a cylinder and an intake manifold, the cylinder including a second exhaust valve in fluid communication with a second exhaust manifold. The method further includes adjusting an amount of fuel injected to the engine in response to an output of a first oxygen sensor positioned in the exhaust system upstream of an emission control device."\nUS10107182B2: "Methods and systems are provided for enhancing turbocharger performance for a boosted engine system configured to operate with a pattern of deactivated cylinders. In one example, a method may include, in response to a demand for boost, operating with a cylinder pattern based on boost demand and turbocharger configuration. The specific pattern may depend on the pattern constraints imposed by engine load and NVH metrics."\nCN106640389B: "Methods and systems are provided for engine speed flare control in an engine system having a multi-stage charge boosting device. In one example, during engine start-up, the upstream compressor is rotated backwards via the electric motor to reduce intake manifold pressure. The engine is then fueled based on the lower manifold pressure to reduce torque and engine speed flare until an idle engine speed condition is reached."\nUS9567892B2: "A cooling system for an internal combustion engine is disclosed. The cooling system includes a radiator for exchanging heat between a coolant and ambient air, and a coolant pump for circulating the coolant. The coolant system further includes a first set of fluid connection branches between the coolant pump and the engine block, the cylinder head and the exhaust manifold and a second set of fluid connection branches between the engine block, cylinder head and exhaust manifold and the radiator and/or the coolant pump. A first controlled valve intercepts the coolant towards the radiator so that the coolant is recirculated towards the coolant pump. A second controlled valve intercepts the coolant from the cylinder block. A third controlled valve intercepts the coolant from the integrated exhaust manifold."\nUS9174637B2: "Methods and systems are provided for improving surge control. When surge conditions are approached, a reference governor reduces engine airflow at a slower rate and to a higher level than the engine airflow required to meet the reduced torque demand. The excess torque resulting from the extra airflow is offset by applying a negative torque on the driveshaft via an electric machine coupled to the engine or via alternate engine actuator adjustments."\nUS7461628B2: "A method of operating an engine, comprising of performing homogeneous charge compression ignition combustion during a first operating condition, and performing spark ignition combustion during a second operating condition, where an amount of directly injected alcohol in at least one of said homogeneous charge compression ignition combustion and said spark ignition combustion is varied in response to at least an operating parameter."\nUS7540264B2: "A method for initializing valves of an engine having a starting apparatus is disclosed. The engine may have electromechanically actuated cylinder valves. The method comprises moving at least a first valve away from a neutral position of the first valve before the engine is rotated by said starting apparatus; and moving at least a second valve away from a neutral position of the second valve after the engine is rotated by said starting apparatus."\nUS5758493A: "A method and apparatus for desulfating a NOx trap is proposed wherein half of the engine cylinders are operated at a rich air/fuel ratio and half are operated at a lean A/F during the desulfation process. The two exhaust gas streams, the rich stream and the lean stream, are physically or chemically separated until they enter the NOx trap. A catalyzed exothermic chemical reaction is then generated in the trap. The resulting temperature increase is sufficient to remove SOX from the trap. During desulfation spark advance is adjusted to minimize event time imbalance."\nUS6338244B1: "A method and apparatus for purifying an exhaust gas of an internal combustion engine includes an ammonia generating catalyst for generating ammonia from nitrogen oxides contained in the exhaust gas to be purified, and a nitrogen oxide reducing catalyst downstream of the ammonia generating catalyst, for reduction of nitrogen oxides contained in the exhaust gas to be purified. Generated ammonia is used as the reducing agent. A nitrogen oxide adsorption catalyst is arranged upstream of the ammonia generating catalyst, or the ammonia generating catalyst is arranged in an exhaust pipe branch which pertains to only a part of several separately controllable internal-combustion sources. One or more additional exhaust pipe branches assigned to the other internal-combustion sources and lead to the nitrogen oxide reducing catalyst, while bypassing the ammonia generating catalyst. The part of the internal-combustion sources pertaining to the exhaust pipe branch of which contains the ammonia generating catalyst is operated either continuously or intermittently in a rich mode. Also, one or more combustion sources which feed their exhaust gas to the nitrogen oxide adsorption catalyst may be operated alternately in a lean and rich mode."\nUS7367189B2: "As the integral value of power obtained by integrating supply power to the three-phase stator coil of an assist motor with respect to time becomes larger, a motor temperature becomes higher. Then, the integral value of power obtained by integrating supply power to the three-phase stator coil of the assist motor with respect to time is detected and when this detected integral value of power is equal to or larger than a first determination value, supply power to the three-phase stator coil of the assist motor is limited. Then, when this detected integral value of power is equal to or larger than a second determination value, supply power to the three-phase stator coil of the assist motor is stopped."\nCN105649809A: "The invention relates to optimizing intermittent fuel pump control. Various methods are provided for operating a fuel pump. In one example, a method of operating a fuel pump comprises iteratively reducing an on-duration of a low pressure fuel pump pulse, until a peak outlet pressure of the fuel pump decreases from a peak outlet pressure corresponding to a previous pulse, to identify a minimum pulse duration, and applying a pulse having the minimum pulse duration to the fuel pump."\n'
""")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [34]:
tru.get_leaderboard(app_ids=["RAG Patent Comparison"])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
RAG Patent Comparison,1.0,0.8,0.7,12.0,0.0
